In [1]:
import pandas as pd
from fuzzywuzzy import process


In [2]:
#import mine data
#primary data
mine_pri = pd.read_excel("/Users/user/Documents/GitHub/Thesis_2425/Data/new coal/Global-Coal-Mine-Tracker-April-2024 (1).xlsx", sheet_name="Global Coal Mine Tracker (Non-C")

In [3]:
#import plant data
plant = pd.read_excel("/Users/user/Documents/GitHub/Thesis_2425/Data/new coal/Global-Coal-Plant-Tracker-January-2025.xlsx", sheet_name="Units")

#correct the estimated capacity with capacity factor
plant['Est_capacity (MW)']=plant['Capacity (MW)']*plant['Capacity factor']

#import terminal data
#note: the capacity of terminal dataset may be overestimated, because some terminals not only receive 'coal' but 'all cargo'
terminal = pd.read_excel("/Users/user/Documents/GitHub/Thesis_2425/Data/new coal/Global-Coal-Terminals-Tracker-December-2024.xlsx", sheet_name="Terminals")

In [4]:
#may further filter by 'status == 'operating'
mine_pri = mine_pri[mine_pri['Country'] == 'India']
plant = plant[plant['Country/Area'] == 'India']
terminal = terminal[terminal['Country/Area'] == 'India']

## Sheet 4: Supply Chain

In [11]:
# 1️⃣ Create supply_chain DataFrame
supply_chain = pd.DataFrame(columns=['supplier', 'client', 'recipe', 'product'])
supply_chain['supplier'] = terminal['GEM Terminal ID'].unique()

# 2️⃣ Filter 'GEM Terminal ID' from terminal data where 'Coal Source' contains different countries
indonesia_terminal_ids = terminal[terminal['Coal Source'].str.contains('Indonesia', case=False, na=False, regex=True)]['GEM Terminal ID'].unique()
australia_terminal_ids = terminal[terminal['Coal Source'].str.contains('Australia', case=False, na=False, regex=True)]['GEM Terminal ID'].unique()
mozambique_terminal_ids = terminal[terminal['Coal Source'].str.contains('Mozambique', case=False, na=False, regex=True)]['GEM Terminal ID'].unique()
south_africa_terminal_ids = terminal[terminal['Coal Source'].str.contains('South Africa', case=False, na=False, regex=True)]['GEM Terminal ID'].unique()

# 3️⃣ Filter 'GEM Plant ID' from plant data where 'Coal Source' contains different countries
indonesia_plant_ids = plant[plant['Coal source'].str.contains('Indonesia', case=False, na=False, regex=True)]['GEM location ID'].unique()
australia_plant_ids = plant[plant['Coal source'].str.contains('Australia', case=False, na=False, regex=True)]['GEM location ID'].unique()
mozambique_plant_ids = plant[plant['Coal source'].str.contains('Mozambique', case=False, na=False, regex=True)]['GEM location ID'].unique()
south_africa_plant_ids = plant[plant['Coal source'].str.contains('South Africa', case=False, na=False, regex=True)]['GEM location ID'].unique()

# 4️⃣ Create a new DataFrame to store the expanded matching relationships
expanded_rows = []

# 5️⃣ Iterate through the filtered terminal IDs and duplicate rows to match multiple plant IDs
for terminal_id in indonesia_terminal_ids:
    for plant_id in indonesia_plant_ids:
        expanded_rows.append({'supplier': terminal_id, 'client': plant_id, 'recipe': '', 'product': ''})

for terminal_id in australia_terminal_ids:
    for plant_id in australia_plant_ids:
        expanded_rows.append({'supplier': terminal_id, 'client': plant_id, 'recipe': '', 'product': ''})

for terminal_id in mozambique_terminal_ids:
    for plant_id in mozambique_plant_ids:
        expanded_rows.append({'supplier': terminal_id, 'client': plant_id, 'recipe': '', 'product': ''})

for terminal_id in south_africa_terminal_ids:
    for plant_id in south_africa_plant_ids:
        expanded_rows.append({'supplier': terminal_id, 'client': plant_id, 'recipe': '', 'product': ''})

# 6️⃣ Convert the list to a DataFrame and append to supply_chain
expanded_df = pd.DataFrame(expanded_rows)

# 7️⃣ Append the expanded data to supply_chain
supply_chain = pd.concat([supply_chain, expanded_df], ignore_index=True)

supply_chain


,supplier,client,recipe,product
0,T1383,NaN,NaN,NaN
1,T1180,NaN,NaN,NaN
2,T1183,NaN,NaN,NaN
3,T1022,NaN,NaN,NaN
4,T1019,NaN,NaN,NaN
...,...,...,...,...
286,T1170,L100000102290,,
287,T1166,L100000102556,,
288,T1166,L100000102290,,
289,T1170,L100000102047,,


In [12]:
# 1️⃣ Create a new DataFrame to store plant data and remove rows where 'Coal Source' contains specific countries
plant_filtered = plant[
    ~plant['Coal source'].str.contains('Indonesia|Australia|South Africa|Mozambique', case=False, na=False, regex=True)
].copy()

# 2️⃣ Filter only rows where 'Coal Source' contains 'mported' (case insensitive)
plant_filtered = plant_filtered[
    plant['Coal source'].str.contains('mported', case=False, na=False, regex=True)
].copy()

# 3️⃣ Identify suppliers (GEM Terminal ID) in supply_chain that do not have a client
suppliers_without_clients = supply_chain[supply_chain['client'].isna()]['supplier'].unique()

# 4️⃣ Retrieve all GEM Plant IDs from the filtered plant data
filtered_plant_ids = plant_filtered['GEM location ID'].unique()

# 5️⃣ For each supplier without a client, duplicate rows and assign all matching GEM Plant IDs
expanded_rows_imported = []
for supplier in suppliers_without_clients:
    for plant_id in filtered_plant_ids:  # Assign all GEM Plant IDs from plant_filtered only to suppliers without clients
        expanded_rows_imported.append({
            'supplier': supplier,
            'client': plant_id,
            'recipe': '',
            'product': ''
        })

# 6️⃣ Convert to a DataFrame and append to supply_chain
expanded_df_imported = pd.DataFrame(expanded_rows_imported)
supply_chain = pd.concat([supply_chain, expanded_df_imported], ignore_index=True)

# 7️⃣ Remove rows where 'client' is still missing and reset the index
supply_chain = supply_chain.dropna(subset=['client']).reset_index(drop=True)

/var/folders/wy/k_hfp9ls0msf61dg9qbm3h040000gn/T/ipykernel_7141/3126874357.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  plant_filtered = plant_filtered[


In [13]:
# 1️⃣ Create four new columns in the supply_chain dataset
supply_chain['supplier_lat'] = None
supply_chain['supplier_lon'] = None
supply_chain['client_lat'] = None
supply_chain['client_lon'] = None

# 2️⃣ Retrieve latitude and longitude for suppliers using GEM Terminal ID from the terminal dataset (remove duplicates)
supplier_coords = terminal[['GEM Terminal ID', 'Latitude', 'Longitude']].drop_duplicates(subset=['GEM Terminal ID']).set_index('GEM Terminal ID')

# 3️⃣ Perform mapping and expand into two separate columns
supplier_mapped = supply_chain['supplier'].map(supplier_coords.to_dict(orient='index'))
supplier_mapped = supplier_mapped.apply(lambda x: pd.Series([x['Latitude'], x['Longitude']]) if isinstance(x, dict) else pd.Series([None, None]))

# 4️⃣ Assign the values to the supply_chain DataFrame
supply_chain[['supplier_lat', 'supplier_lon']] = supplier_mapped

# 5️⃣ Retrieve latitude and longitude for clients using GEM Location ID from the plant dataset (remove duplicates)
client_coords = plant[['GEM location ID', 'Latitude', 'Longitude']].drop_duplicates(subset=['GEM location ID']).set_index('GEM location ID')

# 6️⃣ Perform mapping and expand into two separate columns
client_mapped = supply_chain['client'].map(client_coords.to_dict(orient='index'))
client_mapped = client_mapped.apply(lambda x: pd.Series([x['Latitude'], x['Longitude']]) if isinstance(x, dict) else pd.Series([None, None]))

# 7️⃣ Assign the values to the supply_chain DataFrame
supply_chain[['client_lat', 'client_lon']] = client_mapped

import numpy as np

# Function to calculate the distance between two latitude-longitude points using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth’s radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Compute the distance between each supplier-client pair in the supply_chain dataset
supply_chain['distance_km'] = supply_chain.apply(
    lambda row: haversine(row['supplier_lat'], row['supplier_lon'], row['client_lat'], row['client_lon'])
    if not any(pd.isna([row['supplier_lat'], row['supplier_lon'], row['client_lat'], row['client_lon']])) else np.nan,
    axis=1
)

# Retain only rows where the distance is less than 1000 km
supply_chain = supply_chain[supply_chain['distance_km'] < 1000].reset_index(drop=True)

# Retrieve Coal Type from the plant dataset using client ID (GEM location ID)
client_coal_type = plant[['GEM location ID', 'Coal type']].drop_duplicates(subset=['GEM location ID']).set_index('GEM location ID')

# Map the Coal Type values to the 'product' column in supply_chain
supply_chain['product'] = supply_chain['client'].map(client_coal_type['Coal type'])

# Drop latitude, longitude, and distance columns from supply_chain
supply_chain = supply_chain.drop(columns=['supplier_lat', 'supplier_lon', 'client_lat', 'client_lon', 'distance_km'], errors='ignore')

In [ ]:
excel_path = "/Users/user/Documents/GitHub/Thesis_2425/Data/coal_supply_chain_india.xlsx"  

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    supply_chain.to_excel(writer, sheet_name="supply_chain", index=False)


Not done yet: connect plant data with mine data, but too much data and the result from open ai is also not correct

In [ ]:
plant = plant.dropna(subset=['Coal source']).copy()
plant = plant[
    (plant['Coal source'].str.lower() != 'imported') &
    (plant['Coal source'] != 'Imported coal') &
    (plant['Coal source'] != 'Imported from Indonesia') &
    (plant['Coal source'] != 'Imported (Indonesia)') &
    (plant['Coal source'] != 'Imported Indonesian and Australian coal')
]
mine_pri = mine_pri.drop_duplicates(subset=['GEM Mine ID'], keep='first')


In [ ]:
import time
import openai

# OpenAI API Key
client = openai.OpenAI(api_key='api')

# create supply_chain_mine DataFrame
supply_chain_mine = pd.DataFrame(columns=['supplier', 'client', 'recipe', 'product'])
expanded_rows = []

# iterate mine_pri to match
for _, mine_row in mine_pri.iterrows():
    mine_name = mine_row['Mine Name']
    owner_name = mine_row['Owners']

    for _, plant_row in plant.iterrows():
        coal_source = plant_row['Coal source']

        print(f"Processing: {mine_name} {owner_name} → {coal_source}")

        # prompt
        prompt = f"""
        You are an expert in the Indian coal industry. Your task is to determine whether a given coal source originates from a specific Indian mine or is supplied by a specific Indian owner.
        
        **Given Data:**
        - **Coal Source (from power plant data):** "{coal_source}"
        - **Mine Name (from mine data):** "{mine_name}"
        - **Owner (from mine data):** "{owner_name}"

        **Instructions:**
        - Consider whether the **Coal Source** refers to the **Mine Name** directly or belongs to the **Owner**.
        - If the **Coal Source** clearly refers to the **Mine Name** or **Owner**, return `'Yes'`.
        - If there is no clear relationship, return `'No'`.

        Answer strictly with 'Yes' or 'No' and nothing else.
        """

        # OpenAI API with gpt-3.5-turbo
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            timeout=30  
        )

        match_result = response.choices[0].message.content.strip().lower()
        print(f"🔍 GPT-3.5 结果: {match_result}")

        # if match then store data
        if match_result == "yes":
            expanded_rows.append({
                'supplier': mine_row['GEM Mine ID'],
                'client': plant_row['GEM location ID'],
                'recipe': '',
                'product': plant_row['Coal type']
            })

        time.sleep(0.3)  

supply_chain_mine = pd.concat([supply_chain_mine, pd.DataFrame(expanded_rows)], ignore_index=True)

Processing: Adasa Coal Mine Western Coalfields Ltd → domestic
🔍 GPT-3.5 结果: yes
Processing: Adasa Coal Mine Western Coalfields Ltd → domestic
🔍 GPT-3.5 结果: yes
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → MCL mines and Mandakini Block in Talcher
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → MCL mines and Mandakini Block in Talcher
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → Khadia coal mine extension
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields

KeyboardInterrupt: 

In [ ]:
max_requests = 5 
counter = 0

for _, mine_row in mine_pri.iterrows():
    for _, plant_row in plant.iterrows():
        if counter >= max_requests:
            break
        counter += 1

        print(f"Processing #{counter}: {mine_row['Mine Name']} → {plant_row['Coal source']}")

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            timeout=30
        )

        print(response.choices[0].message.content.strip())


Processing #1: Adasa Coal Mine → imported
No
Processing #2: Adasa Coal Mine → imported
No
Processing #3: Adasa Coal Mine → domestic
No
Processing #4: Adasa Coal Mine → domestic
No
Processing #5: Adasa Coal Mine → South Eastern Coalfields
No
